In [ ]:
import sys
import numpy as np
import random
import pandas as pd
import pickle
import time
import alt_min

In [ ]:
n = 200
r = 20
p = 0.1
eta = 0.1
c = 100

In [74]:
U = np.random.standard_normal((n, r))
V = np.random.standard_normal((r, n))
L = np.matmul(U, V)

In [79]:
row_corruptions = np.zeros(n).astype(int)
col_corruptions = np.zeros(n).astype(int)
mask_eta = np.zeros((n, n))
entries = np.random.permutation(n * n)
corruptions = 0
for entry in entries:
  row = (int)(entry / n)
  col = entry % n
  if row_corruptions[row] < n * eta and col_corruptions[col] < n * eta:
    mask_eta[row][col] = 1
    corruptions += 1
    row_corruptions[row] += 1
    col_corruptions[col] += 1
  if corruptions == n * n * eta:
    break

In [76]:
S = np.random.uniform(-c, c, L.shape)
S *= mask_eta

In [77]:
len(S[S != 0])

3996

In [ ]:
for i in range(n):
  for j in range(n):
    if S[i][j] != 0:
      print((i, j))

In [ ]:
for i in range(n):
  for j in range(n):
    if mask_eta[i][j] != 0:
      print((i, j))

In [ ]:
for i in range(n):
  print(len(mask_eta[i][mask_eta[i] != 0]))

In [ ]:
mask_eta = np.random.rand(L.shape[0], L.shape[1])
S = np.random.uniform(-c, c, L.shape)
S[mask_eta < 1 - eta] = 0

In [63]:
M = L + S
mask_p = np.random.rand(M.shape[0], M.shape[1])
mask_p[mask_p < 1 - p] = 0
mask_p[mask_p != 0] = 1
M_obs = np.multiply(M, mask_p)

In [64]:
len(mask_p[mask_p != 0])

3989

In [ ]:
user_col = []
item_col = []
rating_col = []
for i in range(M_obs.shape[0]):
  for j in range(M_obs.shape[1]):
    rating = M_obs[i][j]
    if rating != 0:
      user_col.append(i)
      item_col.append(j)
      rating_col.append(rating)
train = pd.DataFrame.from_dict({"user": user_col, "item": item_col, "rating": rating_col})

In [ ]:
user_col = []
item_col = []
rating_col = []
for i in range(L.shape[0]):
  for j in range(L.shape[1]):
    rating = L[i][j]
    user_col.append(i)
    item_col.append(j)
    rating_col.append(rating)
test = pd.DataFrame.from_dict({"user": user_col, "item": item_col, "rating": rating_col})

In [ ]:
model = alt_min.LeastSquares(train, test, 2000, reg=0)
model.alt_min()

In [ ]:
train.loc[0][0]

In [ ]:
train["user"]

In [ ]:
n = 10000
d = 30
eta = 0.1
X = np.random.standard_normal((n, d))
corrupt_idx = np.random.choice(n, (int)(eta * n), replace=False)
X[corrupt_idx] = np.random.uniform(0, 1000, (len(corrupt_idx), d))
est = alt_min.agnosticMeanGeneral(X, eta)
index_array = np.argsort(np.linalg.norm(X, axis=1))
robust_mean = np.mean(X[index_array][:(int)((1 - eta) * X.shape[0])], axis=0)
print("mean norm: {}".format(np.linalg.norm(np.mean(X, axis=0))))
print("median norm: {}".format(np.linalg.norm(np.median(X, axis=0))))
print("est norm: {}".format(np.linalg.norm(est)))
print("robust norm: {}".format(np.linalg.norm(robust_mean)))

In [ ]:
robust_mean.shape

In [ ]:
index_array

In [ ]:
X

In [ ]:
X[index_array][:(int)((1 - eta) * X.shape[0])]

In [ ]:
np.linalg.norm(robust_mean)

In [ ]:
robust_mean

In [ ]:
np.linalg.norm(X, axis=1).astype(int)